In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_experiments
exp_name = '2020_12_18_flying'; 
fname = f'results/{exp_name}_real.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

In [ ]:
df_total.tail()

# Preprocessing

## a. compute distance estimates

TODO: this part is hacky and will be improved by getting imu estimates rather than flow dx/dy estimates, which turn out to be very unreliable.

In [ ]:
MIN_S = 5 # read from plot
MAX_S = 25 # read from plot

def add_distance_estimates(df_total, i, ax=None, min_time=MIN_S, max_time=MAX_S, min_z=300):
    row = df_total.loc[i]
    row.z[np.isnan(row.z)] = 0
    start_idx = np.where(row.seconds > min_time)[0][0]
    end_idx = np.where(row.seconds < max_time)[0][-1]

    assert end_idx > start_idx, f"{start_idx}<={end_idx}"
    valid_z = row.z[start_idx:end_idx]
    max_idx = start_idx + np.argmax(valid_z)
    min_idx = start_idx + np.where(valid_z > min_z)[0][-1]
    duration = row.seconds[min_idx] - row.seconds[max_idx]

    d = np.full(len(row.seconds), np.nan)
    times = row.seconds[max_idx:min_idx] - row.seconds[max_idx]
    d[max_idx:min_idx] = D_START - times * VELOCITY
    df_total.loc[i, 'd_estimate'] = d
    
    if ax is not None:
        ax.axvline(row.seconds[start_idx], color='r')
        ax.axvline(row.seconds[max_idx], color='k', label='start (max)')
        ax.axvline(row.seconds[min_idx], color='k', label=f'end (above {min_z})')
        ax.axvline(row.seconds[end_idx], color='b')
        ax.set_xlim(0, 35)
        ax.scatter(row.seconds, d * 1000, color='k', label='distance')
        ax.set_title(f"duration: {duration:.1f}s")
        ax.legend()

In [ ]:
VELOCITY = 0.05 # [m/s], parameter in crazyflie
D_START = 0.6 # [m], starting distance 

df_total = df_total.assign(d_estimate=None) # initialize df_estimate
for i, row in df_total.iterrows():
    fig, ax = plt.subplots()
    ax.plot(row.positions[:, 0], label='x', color='C0')
    ax.plot(row.positions[:, 1], label='y', color='C1')
    ax.plot(row.positions[:, 2], label='z', color='C2')
    #ax.plot(row.dx)
    #ax.plot(row.dy)
    ax.set_ylim(-10, 50)
    #ax.set_xlim(0, 1000)
    ax.set_xlabel('position idx [-]')
    ax.set_ylabel('x, y [mm]')
    ax.legend(loc='lower left')
    ax.set_title("lateral movement, " + row.source + row.appendix)
    
    plt.figure()
    #plt.plot(row.yaw_deg)
    plt.plot(row.seconds, row.z, color='C2')
    plt.xlabel('time [s]')
    plt.ylabel('z [mm]')
    #plt.ylim(-90, 90)
    plt.ylim(0, 650)
    #plt.xlim(MIN_S, MAX_S)
    
    add_distance_estimates(df_total, i, ax=plt.gca())

## b. compute spectrogram

In [ ]:
def add_spectrogram(row):
    # spectrogram is of shape 1025 x n_times x n_mics
    spectrogram = np.zeros((len(all_frequencies), row.signals_f.shape[0], row.signals_f.shape[1]), dtype=float) 
    for t_idx in range(row.signals_f.shape[0]):
        freqs = row.frequencies_matrix[t_idx, :]
        for i, f in enumerate(freqs):
            f_idx =  np.argmin(np.abs(f - all_frequencies))
            if abs(all_frequencies[f_idx] - f) > 1: 
                print(f'Warning: frequency {f} is far from {all_frequencies}')
            if 0: #np.any(spectrogram[f_idx, t_idx, :] > 0):
                print('overwriting', t_idx, f_idx, f, freqs[0])
                err = np.max(np.abs(np.abs(row.signals_f[t_idx, :, i]) - spectrogram[f_idx, t_idx, :]))
                # TODO(FD) find out why there is a difference between the
                # first freq. bin (forced) and the one selected by snr scheme. 
                if err > 0.1:
                    print('big error:', err)
            spectrogram[f_idx, t_idx, :] = np.abs(row.signals_f[t_idx, :, i])
    row.spectrogram = spectrogram
    return row

def plot_spectrogram(ax, row, min_freq, max_freq, mic_idx=0):
    plot_freq = (all_frequencies > min_freq) & (all_frequencies < max_freq)
    plot_times = ~np.isnan(row.d_estimate)
    #print('time window with valid distances:', np.where(plot_times)[0][[0, -1]])
    spec = row.spectrogram[plot_freq, :, mic_idx][:-1, :]
    spec = spec[:, plot_times][:, :-1]
    
    spec[spec > 0] = np.log10(spec[spec > 0])
    spec[spec == 0] = np.nan
    
    ax.pcolorfast(row.seconds[plot_times], all_frequencies[plot_freq], spec)

In [ ]:
from crazyflie_description_py.parameters import N_BUFFER, FS
all_frequencies = np.round(np.fft.rfftfreq(N_BUFFER, 1/FS), 0)
df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

# Analysis

## 0. Propeller noise study

In [ ]:
from wall_analysis import filter_by_dicts
sys.path.append(f'../experiments/{exp_name}/')
from params import FREQS
appendix = "_new"
source = "None"

df_sources = filter_by_dicts(df_total, [{"source":source, "appendix":appendix}])
assert len(df_sources) == 1
row = df_sources.iloc[0]

n_mics = row.signals_f.shape[1]
times_idx = ~np.isnan(row.d_estimate)
distances = row.d_estimate[times_idx] * 100

In [ ]:
import scipy.signal
fig, axs = plt.subplots(1, n_mics, sharex=True)#, sharey=True)
fig.set_size_inches(20, 5)

min_freq = 100
max_freq = 5000
spec = row.spectrogram[:, times_idx, :]
for i in range(n_mics):
    #fig, ax = plt.subplots()
    #fig.set_size_inches(10, 5)
    plot_spectrogram(axs[i], row, min_freq=min_freq, max_freq=max_freq, mic_idx=i)
    [axs[i].axhline(f, label=f'{f}', color=f'C{j}', ls=':') for j, f in enumerate(FREQS)]
    axs[i].set_title(f'mic{i}')
    axs[i].set_xlabel(f'seconds [s]')
fig.suptitle('spectrograms during wall approach')
    
    
plt.figure()
mask = (all_frequencies > min_freq) & (all_frequencies < max_freq)
spec_average = np.mean(np.mean(spec[mask, :, :], axis=2), axis=1)

max_indices = scipy.signal.find_peaks(spec_average, height=5)[0][:3]
#max_indices = np.argsort(spec_average)[::-1][:3]
plt.plot(all_frequencies[mask], spec_average)
[plt.axvline(all_frequencies[mask][m], color=f'C{j}', label=f'peak{j}', ls=':') for j, m in enumerate(max_indices)]
plt.grid(which='both')
plt.legend()

In [ ]:
#fig, axs = plt.subplots(len(max_indices), n_mics, sharex=True, squeeze=False, sharey=True)

n_harmonics = 4
fig, axs = plt.subplots(n_harmonics, n_mics, sharex=True, squeeze=False)#, sharey=True)
fig.set_size_inches(20, 10)

# choose only times where the max snr bin is given. 
#freqs = all_frequencies[mask][max_indices]
freq_max = 703
times_idx = np.where(row.frequencies_matrix[:, 1] == freq_max)[0]
distances = row.d_estimate[times_idx] * 100

#for j, f_idx in enumerate(max_indices):
for j in range(n_harmonics):
    freq = freq_max * (j + 1)
    print(freq)
    f_idx = np.where(all_frequencies[mask] == freq)[0][0]
    #freq = int([f_idx])
    spec = row.spectrogram[mask, :, :]
    
    for i in range(n_mics):
        spec_here = spec[f_idx, times_idx, i]
        #axs[j, i].plot(row.seconds[times_idx], spec, color=f'C{j}', label=f'{f}')
        axs[j, i].plot(distances, spec_here, color=f'C{j}', label=f'{freq:.0f}Hz')
        axs[j, i].legend(loc='upper left')
        axs[j, i].set_title(f'mic{i}')
    #[axs[0, i].set_xlim(5, 20) for i in range(n_mics)]
    #[axs[0, i].set_xlim(5, 60) for i in range(n_mics)]
    [axs[-1, i].set_xlabel('distance [cm]') for i in range(n_mics)]
    pass

## 1. frequency selection

Investigate 
- what bins are chosen by forced scheme (black dots below)
- when spectrogram at given bin is nonzero 

Above suggests that even when we do not play the given bins, there is significant propeller noise there (thus they are chosen among the 32 strongest frequency bins). The only exception is bin 2375, which is very clean.

In [ ]:
print('available sources:', df_total.source.unique())
filter_dict = {
    'source': 'None',
    #'source': 'mono1750',
    #'source': 'mono2375',
    #'source': 'mono3125',
    #'source': 'mono3875',
    #'source': 'sweep_slow',
    #'source': 'sweep_fast',
    'appendix': '_new'
}
rows = filter_by_dicts(df_total, [filter_dict])
assert len(rows) == 1
row = rows.iloc[0]

In [ ]:
for source in [s for s in df_total.source.unique() if 'mono' in s]:
    filter_dict['source'] = source
    rows = filter_by_dicts(df_total, [filter_dict])
    assert len(rows) == 1
    row = rows.iloc[0]
    
    for mic_idx in [2]:
        fig, ax = plt.subplots()
        fig.set_size_inches(10, 5)

        plot_spectrogram(ax, row, min_freq=100, max_freq=5000, mic_idx=mic_idx)
        [ax.axhline(f, label=f'{f}', color=f'C{i}', ls=':') for i, f in enumerate(FREQS)]
        ax.set_title(row.source + row.appendix + f" mic{mic_idx}")
        ax.legend(loc='lower left')

In [ ]:
strongest_bin = row.frequencies_matrix[:, 0]
fig = plt.figure()
fig.set_size_inches(10, 5)
plt.scatter(row.seconds, strongest_bin, color='k', label='forced')

#mic_indices = [0]
mic_indices = range(n_mics)

for i, f in enumerate(FREQS):
    plt.axhline(f, label=f'{f}', color=f'C{i}')
    
    # find out where the spectrogram is non-zero for this frequency bin. 
    f_idx = np.where(f == all_frequencies)[0][0]
    
    spec_total = np.mean(row.spectrogram[f_idx, :, mic_indices], axis=0)
    spec_total -= np.min(spec_total)
    spec_total *= 50 #/ np.max(spec_total)
    times = row.seconds[spec_total > 0]
    plt.scatter(times, f + spec_total[spec_total > 0], color=f'C{i}')
    plt.legend()
    if len(times) > 0:
        plt.xlim(times[0], 18)
    #plt.xlim(times[0], times[-1])

## 2. interference studies

In [ ]:
appendix = "_new"

freqs = FREQS

fig, axs = plt.subplots(len(freqs), n_mics, sharex=True)#, sharey=True)
fig.set_size_inches(20, 20)
for j, f in enumerate(freqs): 
    f_idx = np.where(f == all_frequencies)[0][0]
    
    source = f"mono{f}"
    df_sources = filter_by_dicts(df_total, [{"source":source, "appendix":appendix}])
    assert len(df_sources) == 1
    row = df_sources.iloc[0]
    
    # forced
    #times_idx = np.where(f == row.frequencies_matrix[:, 0])[0]
    # automatic
    #times_idx = np.where(np.sum(spectrogram[f_idx, :, :], axis=1) > 0)[0]
    times_idx = ~np.isnan(row.d_estimate)
    distances = row.d_estimate[times_idx] * 100
    
    for i in range(n_mics):
        spec = row.spectrogram[f_idx, times_idx, i]
        #axs[j, i].plot(row.seconds[times_idx], spec, color=f'C{j}', label=f'{f}')
        axs[j, i].plot(distances, spec, color=f'C{j}', label=f'{f} Hz')
        axs[j, i].legend(loc='upper left')
        axs[j, i].set_title(f'mic{i}')
#[axs[0, i].set_xlim(5, 20) for i in range(n_mics)]
#[axs[0, i].set_xlim(5, 60) for i in range(n_mics)]
[axs[-1, i].set_xlabel('distance [cm]') for i in range(n_mics)]
pass

In [ ]:
freqs = FREQS

sources = ['sweep_fast', 'sweep_slow']
appendix = "_new"
for source in sources:
    
    df_sources = filter_by_dicts(df_total, [{"source":source, "appendix":appendix}])
    for i, row in df_sources.iterrows():
        
        fig, axs = plt.subplots(len(freqs), n_mics, sharex=True)#, sharey=True)
        fig.set_size_inches(20, 20)
        fig.suptitle(row.source + row.appendix, y=0.9)
        for j, f in enumerate(freqs): 
            f_idx = np.where(f == all_frequencies)[0][0]

            # forced
            times_idx = np.where(f == row.frequencies_matrix[:, 0])[0]
            # automatic
            #times_idx = np.where(np.sum(spectrogram[f_idx, :, :], axis=1) > 0)[0]
            #times_idx = ~np.isnan(row.d_estimate)
            distances = row.d_estimate[times_idx] * 100

            for i in range(n_mics):
                spec = row.spectrogram[f_idx, times_idx, i]
                #axs[j, i].plot(row.seconds[times_idx], spec, color=f'C{j}', label=f'{f}')
                axs[j, i].plot(distances, spec, color=f'C{j}', label=f'{f}', marker='x')
                axs[j, i].legend(loc='upper left')
                axs[j, i].set_title(f'mic{i}')
        #[axs[0, i].set_xlim(5, 20) for i in range(n_mics)]